In [1]:
import os
os.chdir("/fast-depth")
print(os.getcwd())

/fast-depth


In [2]:
from PIL import Image
import numpy as np
from imagenet.mobilenet import MobileNet
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm

# Hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 0.0025

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Datasets
train_dataset = datasets.ImageFolder(root="/scratchdata/cat_dog/training_set/training_set/", transform=transform)
val_dataset = datasets.ImageFolder(root="/scratchdata/cat_dog/test_set/test_set/", transform=transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)


In [4]:
class Classfier(nn.Module):
    def __init__(self):
        super(Classfier, self).__init__()
        self.fc = nn.Linear(1000, 2)

    def forward(self, x):
        x = self.fc(x)
        #x = nn.functional.softmax(x, dim=1)
        return x

In [5]:
model = nn.Sequential(MobileNet(), Classfier()).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='it') as pbar:
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)

            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Update the progress bar
            pbar.update(1)  # Increment the progress bar
            pbar.set_postfix(loss=loss.item())  # Display the loss

    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')



Epoch 1/10: 100%|██████████| 251/251 [01:30<00:00,  2.77it/s, loss=0.592]


Validation Accuracy: 64.16%


Epoch 2/10: 100%|██████████| 251/251 [00:47<00:00,  5.26it/s, loss=0.639]


Validation Accuracy: 69.20%


Epoch 3/10: 100%|██████████| 251/251 [00:47<00:00,  5.28it/s, loss=0.273]


...

Validation Accuracy: 84.97%


Epoch 9/10: 100%|██████████| 251/251 [00:48<00:00,  5.14it/s, loss=0.51]  


Validation Accuracy: 86.51%


Epoch 10/10: 100%|██████████| 251/251 [00:48<00:00,  5.21it/s, loss=0.558] 


Validation Accuracy: 84.58%


In [6]:
# Save the model checkpoint as pth file
torch.save(model.state_dict(), 'model.pth')